In [1]:
using AbstractGPs
using TemporalGPs
using Distributions
using StatsFuns: logistic

In this example we are showing how to work with non-Gaussian likelihoods,
here a binary classification problem with a logistic link,
by using variable augmentations as in [Galy-Fajou et al, 2020](https://arxiv.org/abs/2002.114510).
We base our example on the "exact_time_inference" example

Load up the separable kernel from TemporalGPs.

In [2]:
using TemporalGPs: RegularSpacing

Build a GP as per usual, and wrap it inside a TemporalGPs.jl object.

In [3]:
f_raw = GP(Matern52Kernel());
f = to_sde(f_raw, SArrayStorage(Float64));

Specify a collection of inputs. Must be increasing.

In [4]:
T = 1_000;
x = RegularSpacing(0.0, 1e-1, T);

Generate some synthetic data from the GP and get a random binary output

In [5]:
σ²_noise = 0.01;
f_true = rand(f(x, σ²_noise));
y = rand.(Bernoulli.(logistic.(f_true)));
y_sign = sign.(y .- 0.5)

1000-element Vector{Float64}:
  1.0
  1.0
 -1.0
 -1.0
 -1.0
  1.0
  1.0
 -1.0
 -1.0
 -1.0
  ⋮
 -1.0
  1.0
  1.0
  1.0
 -1.0
  1.0
  1.0
 -1.0
  1.0

We are using the augmentation trick and look for the optimal
expected value of the variable ω

In [6]:
η₁_like(ω, g, β) = g .+ β .* ω # The augmented likelihood first natural parameter
Λ_like(ω, γ) = 2 * γ * ω # The augmented likelihood precision (diagonal)

Λ_like (generic function with 1 method)

These are the values corresponding to our problem

In [7]:
γ = 0.5
g = 0.5 * y_sign
β = 0

function compute_optimal_expectation(f, x, g, β, γ; n_iter=5)
    T = length(x)
    ω̄ = rand(T) # preallocation
    c = zeros(T) # preallocation
    for i in 1:n_iter
        Λ = Λ_like(ω̄, γ)
        p_f = marginals(posterior(f(x, inv.(Λ)), inv.(Λ) .* η₁_like(ω̄, g, β))(x))
        @. c = sqrt(var(p_f) + abs2(mean(p_f))) / 2
        @. ω̄ = 0.5 * tanh(c) / c
    end
    return ω̄
end

ω̄ = compute_optimal_expectation(f, x, g, β, γ)
Λ = Λ_like(ω̄, γ)
f_post = posterior(f(x, inv.(Λ)), inv.(Λ) .* η₁_like(ω̄, g, β))

TemporalGPs.PosteriorLTISDE{TemporalGPs.LTISDE{AbstractGPs.GP{AbstractGPs.ZeroMean{Float64}, KernelFunctions.Matern52Kernel{Distances.Euclidean}}, TemporalGPs.SArrayStorage{Float64}}, NamedTuple{(:y, :x, :Σy), Tuple{Vector{Float64}, TemporalGPs.RegularSpacing{Float64}, LinearAlgebra.Diagonal{Float64, Vector{Float64}}}}}(TemporalGPs.LTISDE{AbstractGPs.GP{AbstractGPs.ZeroMean{Float64}, KernelFunctions.Matern52Kernel{Distances.Euclidean}}, TemporalGPs.SArrayStorage{Float64}}(AbstractGPs.GP{AbstractGPs.ZeroMean{Float64}, KernelFunctions.Matern52Kernel{Distances.Euclidean}}(AbstractGPs.ZeroMean{Float64}(), Matern 5/2 Kernel (metric = Distances.Euclidean(0.0))), TemporalGPs.SArrayStorage{Float64}()), (y = [1.0253678314631345, 1.0217438141783033, -1.0195677020527874, -1.0189552559481503, -1.0198383362582495, 1.0220265589909046, 1.0252356506747304, -1.0290306914103913, -1.0327896972200443, -1.0358026810522216  …  1.042910466104591, -1.0415497599450303, 1.0393642849364477, 1.036776928910973, 1.

Specify some locations at which to make predictions.

In [8]:
T_pr = 1_200_000;
x_pr = RegularSpacing(0.0, 1e-4, T_pr);

Compute the exact posterior marginals at `x_pr`.

In [9]:
f_post_marginals = marginals(f_post(x_pr));
m_post_marginals = mean.(f_post_marginals);
σ_post_marginals = std.(f_post_marginals);

Generate a few posterior samples. Not fantastically-well optimised at present.

In [10]:
f_post_samples = [rand(f_post(x_pr)) for _ in 1:5];

Visualise the posterior. The if block is just to prevent it running in CI.

In [11]:
if get(ENV, "TESTING", "FALSE") == "FALSE"
    using Plots
    plt = plot()
    plot!(plt, f_post(x_pr); ribbon_scale=3.0, label="");
    plot!(plt, x_pr, f_post_samples; color=:red, alpha=0.3, label="");
    plot!(plt, x, f_true; label="", lw=2.0, color=:blue); # Plot the true latent GP on top
    scatter!(plt, x, y; label="", markersize=1.0, alpha=1.0); # Plot the data
    savefig(plt, "posterior.png");
end

"/home/runner/work/TemporalGPs.jl/TemporalGPs.jl/docs/src/examples/augmented_inference/posterior.png"

<hr />
<h6>Package and system information</h6>
<details>
<summary>Package information (click to expand)</summary>
<pre>
Status &#96;~/work/TemporalGPs.jl/TemporalGPs.jl/examples/augmented_inference/Project.toml&#96;
  &#91;99985d1d&#93; AbstractGPs v0.5.16
  &#91;31c24e10&#93; Distributions v0.25.87
  &#91;98b081ad&#93; Literate v2.14.0
  &#91;91a5bcdd&#93; Plots v1.38.9
  &#91;4c63d2b9&#93; StatsFuns v1.3.0
  &#91;e155a3c4&#93; TemporalGPs v0.6.3 &#96;/home/runner/work/TemporalGPs.jl/TemporalGPs.jl#564e73f&#96;
</pre>
To reproduce this notebook's package environment, you can
<a href="./Manifest.toml">
download the full Manifest.toml</a>.
</details>
<details>
<summary>System information (click to expand)</summary>
<pre>
Julia Version 1.8.5
Commit 17cfb8e65ea &#40;2023-01-08 06:45 UTC&#41;
Platform Info:
  OS: Linux &#40;x86_64-linux-gnu&#41;
  CPU: 2 × Intel&#40;R&#41; Xeon&#40;R&#41; CPU E5-2673 v4 @ 2.30GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 &#40;ORCJIT, broadwell&#41;
  Threads: 1 on 2 virtual cores
Environment:
  JULIA_DEBUG &#61; Documenter
  JULIA_LOAD_PATH &#61; :/home/runner/.julia/packages/JuliaGPsDocs/e8FS0/src
</pre>
</details>

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*